This notebook computes all of the "distances" (the fuzzy logic scores for IOU, DICE, ...) between the GradCam maps produced by the final resNet50/efficientnet models and the aggregated visual characteristics maps created by the derms. That is: 

```
for each image i  
  for each GradCam image i_gc  
    for each derm d  
      agg_map = create_aggregated_characteristics_map()      
      compute the visual distance between i_gc and agg_map 
```


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
from matplotlib.pyplot import imshow
from PIL import Image


In [63]:
# Set paths

# Path to the GradCam images.
gradcam_main_path = Path('/home/ubuntu/store/efficientnet-final-size/visualisation/gradcam/')

# Set path to the model used to create the visualisations.
model_path = Path('efficientnetb4_4')

# Path to the masks create by the derms for the individual characteristics.
derm_mask_main_path = Path('/home/ubuntu/store/masks/masks_resized')

# Path to the rescaled test images.
test_im_path = Path('/home/ubuntu/store/DermX-test-set/test/')


In [64]:
full_gradcam_path = gradcam_main_path / model_path


In [65]:
classes = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo"
]

derms = [
    'derm0',
    'derm1',
    'derm2',
    'derm3',
    'derm4',
    'derm5',
    'derm6',
    'derm7',
]

chars = [
    'closed-comedo',
    'cyst',
    'dermatoglyph-disruption',
    'leukotrichia',
    'macule',
    'nodule',
    'open-comedo',
    'papule',
    'patch',
    'plaque',
    'pustule',
    'scale',
    'scar',
    'sun-damage',
    'telangiectasia',
    'thrombosed-capillaries'
]


In [ ]:
# Extract the paths to the derm masks. The resulting list is only used for a sanity check.
derm_mask_paths = [p for p in derm_mask_main_path.iterdir() if p.suffix == '.png']
derm_mask_paths[0:5]

In [ ]:
# Extract the paths to the test images. Only needed for visualization and debugging.
test_img_paths = [p for p in Path(test_im_path).rglob('*.jpeg')]
test_img_paths[0:5]

In [68]:
# Metrics supporting probabilistic segmentation maps
fuzzy_and = lambda x,y: np.minimum(x,y)
fuzzy_or = lambda x,y: np.maximum(x,y)
fuzzy_not = lambda x: 1-x

def pixel_metrics_fuzzy(y_true, y_pred):
    """
    Pixel-level metrics of segmentation accuracy following fuzzy logic operators.
    
    :param y_true: numpy.ndarray of reference segmentation, values in [0,1]
    :param y_pred: numpy.ndarray of predicted segmentation, values in [0,1]

    :return: a dictionary encoding the metrics
    """
        
    np.testing.assert_equal(y_true.shape, y_pred.shape, err_msg="Expecting \
    the reference and predicted segmentations to be of the same size.")
    
    # Check the ranges
    np.testing.assert_equal(np.logical_and(y_true >= 0, y_true <= 1).all(), True, err_msg="Expecting \
    the reference segmentations to be in the range 0 to 1.")
    np.testing.assert_equal(np.logical_and(y_pred >= 0, y_pred <= 1).all(), True, err_msg="Expecting \
    the predicted segmentations to be in the range 0 to 1.")
    
    TP = fuzzy_and(y_true, y_pred).sum()
    TN = fuzzy_and(fuzzy_not(y_true), fuzzy_not(y_pred)).sum()
    union = fuzzy_or(y_true, y_pred).sum()
    
    metrics = {}
    
    # Summary metrics
    metrics["iou"] = TP / union
    metrics["dice"] = 2 * TP / ( y_true.sum() + y_pred.sum() ) 
    
    # Positive class metrics
    metrics["precision"] = TP / y_pred.sum()
    metrics["recall"] = TP / y_true.sum()
    
    # Negative class metrics
    metrics["negative_predictive_value"] = TN / fuzzy_not(y_pred).sum()
    metrics["specificity"] = TN / fuzzy_not(y_true).sum()
    
    return metrics

In [69]:
def create_aggregated_characteristics_map(mask_path, chars, mask_size):
    """
    Combine all characteristics maps made by a given derm on a given image, into a single "activation map".
    
    Input:
    mask_path: Partial path to the char maps. Only the name of the char and the file extension is missing.
    chars: List of characteristics
    mask_size: tuple with the size of the output maps. The format is (rows, cols)
    
    Output:
    A binary numpy array of size "mask_size". 
    
    """
    agg_mask = np.zeros(mask_size)
    for ch in chars:
        # Check if this derm has created a mask for this characteristic.
        full_mask_path = Path(str(mask_path) + ch + '.png')
        if full_mask_path.is_file():
            mask_im = Image.open(full_mask_path)
            mask_im = mask_im.resize((mask_size[1], mask_size[0]), Image.NEAREST)
            agg_mask = np.logical_or(agg_mask, np.asarray(mask_im) > 0)
    
    return agg_mask
    

In [70]:
# For each image in the test set, calculate the value of the defined metrics given the GradCam image for a given
# class and the "derm activation map" created by aggregating all of the characteristics outlined by a specific derm.
# The result is stored in a defaultdict. The aggregated map is created using logical 'or' between the characteristics
# maps.
rec_dd = lambda: defaultdict(rec_dd)
out_gradcam = rec_dd()

for p in test_img_paths:
    for c in classes:
        # Check if matching GradCam file exists.
        gc_path = full_gradcam_path / Path(p.stem + '_' + c + '.npy')
        if gc_path.is_file():
            for d in derms:
                # Check if this derm has made any masks for this image. If so compute the aggregated map, otherwise
                # skip.
                make_agg = False
                for ch in chars:
                    mask_path = derm_mask_main_path / Path(p.stem + '_' + d + '_2021-05-27-masks_' + ch + '.png') 
                    if mask_path.is_file():
                        make_agg = True
                if make_agg:
                    partial_mask_path = derm_mask_main_path / Path(p.stem + '_' + d + '_2021-05-27-masks_')
                    agg_ch_map = create_aggregated_characteristics_map(partial_mask_path, chars, (300,400))
                    
                    # Calculate the value of the metics given the GradCam image and the aggregated derm mask.
                    gradcam_im = np.load(gc_path, allow_pickle=True)
                    out_gradcam[p.stem][c][d] = pixel_metrics_fuzzy(agg_ch_map, gradcam_im)
        else:
            print('GradCam file missing for image: ', p)


In [72]:
def transform_to_list(nested_dict):
    # Transform the gradcam defaultdict to a list of tuples.
    
    out = []
    for im_name, class_dict in nested_dict.items():
        for class_name, derm_dict in class_dict.items():
            for derm_name, metric_dict in derm_dict.items():
                    tmp = tuple(metric_dict.values())
                    out.append( (im_name, class_name, derm_name) + tmp )
    return out

In [73]:
gradcam_res_list = transform_to_list(out_gradcam)
len(gradcam_res_list)

8892

### Make output dataFrames/csv files

In [75]:
col_names = ['image_name',
             'visualisation_class',
             'derm',
             'iou',
             'dice',
             'precision',
             'recall',
             'negative_predictive_value',
             'specificity'
            ]

In [ ]:
gradcam_df = pd.DataFrame.from_records(gradcam_res_list, columns=col_names)
gradcam_df.head(4)

In [77]:
# Save dataFrames
model_name = str(model_path)
gradcam_df.to_csv(model_name + "_aggregated_derm_maps_vs_gradcam_scores.csv")



# Filter

In [78]:
def get_table(results_pred):
    results_mean = results_pred.groupby('visualisation_class').mean()
    results_std =  results_pred.groupby('visualisation_class').std()
    
    ### Add means
    results_mean.loc['mean'] = results_mean.mean()
    results_std.loc['mean']   = results_mean.mean()
    
    columns = results_mean.columns.to_list()
    for col in columns:
        results_mean[col] = results_mean[col].apply(lambda x: f'\${np.round(x,decimals=2)} \pm ')
        results_std[col] = results_std[col].apply(lambda x: f'\${np.round(x,decimals=2)}')
    
    table_pred = results_mean + results_std  
    return table_pred

In [79]:
class_map = {'0' : 'Acne' ,
             '1' : 'Actinic keratosis',
             '2' : 'Psoriasis' ,
             '3' : 'Seborrheic dermatitis',
             '4' : 'Viral warts',
             '5' : 'Vitiligo'}

filefolder = model_name + "_aggregated_derm_maps_vs_gradcam_scores.csv"
predsfile = '/home/ubuntu/store/efficientnet-final-size/' + model_name + '_preds.csv'

In [ ]:
df_preds = pd.read_pickle(predsfile)
gradcam_df = pd.read_csv(filefolder)
gradcam_df=gradcam_df.drop(['Unnamed: 0'], axis=1)
gradcam_df.head(4)

In [ ]:
df_preds['pred'] =  df_preds['pred'].apply(lambda x: class_map[str(x)])
df_preds['actual'] = df_preds['actual'].apply(lambda x: class_map[str(x)])
df_preds['filenames'] = df_preds['filenames'].apply(lambda x: Path(x).stem)
df_preds = df_preds.rename(columns={'filenames':'image_name'})
df_preds.head(4)

In [22]:
result = pd.merge(gradcam_df,df_preds, on ="image_name")
results_pred = result[result.visualisation_class == result.pred]
table_pred = get_table(results_pred)
# Get it for actual
results_actual = result[result.visualisation_class == result.actual  ]
table_actual = get_table(results_actual)

table_pred.to_pickle('./' + model_name + '_aggregated_derm_maps_vs_gradcam_scores_pred.pkl')
table_actual.to_pickle('./' + model_name + '_aggregated_derm_maps_vs_gradcam_scores_actual.pkl')


In [24]:
table_pred

,iou,dice,precision,recall,negative_predictive_value,specificity
visualisation_class,,,,,,
Acne,\$0.12 \pm \$0.1,\$0.19 \pm \$0.15,\$0.22 \pm \$0.22,\$0.3 \pm \$0.19,\$0.83 \pm \$0.18,\$0.81 \pm \$0.1
Actinic keratosis,\$0.29 \pm \$0.12,\$0.43 \pm \$0.16,\$0.63 \pm \$0.34,\$0.41 \pm \$0.13,\$0.6 \pm \$0.29,\$0.86 \pm \$0.07
Psoriasis,\$0.23 \pm \$0.13,\$0.35 \pm \$0.18,\$0.41 \pm \$0.27,\$0.42 \pm \$0.19,\$0.77 \pm \$0.21,\$0.8 \pm \$0.1
Seborrheic dermatitis,\$0.16 \pm \$0.1,\$0.26 \pm \$0.15,\$0.23 \pm \$0.19,\$0.52 \pm \$0.14,\$0.83 \pm \$0.17,\$0.61 \pm \$0.09
Viral warts,\$0.26 \pm \$0.09,\$0.4 \pm \$0.12,\$0.43 \pm \$0.22,\$0.48 \pm \$0.16,\$0.79 \pm \$0.18,\$0.81 \pm \$0.07
Vitiligo,\$0.13 \pm \$0.09,\$0.21 \pm \$0.13,\$0.16 \pm \$0.12,\$0.52 \pm \$0.24,\$0.93 \pm \$0.09,\$0.74 \pm \$0.08
mean,\$0.2 \pm \$0.2,\$0.31 \pm \$0.31,\$0.35 \pm \$0.35,\$0.44 \pm \$0.44,\$0.79 \pm \$0.79,\$0.77 \pm \$0.77


In [25]:
table_actual

,iou,dice,precision,recall,negative_predictive_value,specificity
visualisation_class,,,,,,
Acne,\$0.1 \pm \$0.1,\$0.17 \pm \$0.15,\$0.21 \pm \$0.22,\$0.24 \pm \$0.2,\$0.82 \pm \$0.19,\$0.83 \pm \$0.1
Actinic keratosis,\$0.26 \pm \$0.12,\$0.39 \pm \$0.16,\$0.6 \pm \$0.34,\$0.41 \pm \$0.15,\$0.64 \pm \$0.27,\$0.87 \pm \$0.08
Psoriasis,\$0.3 \pm \$0.11,\$0.45 \pm \$0.14,\$0.49 \pm \$0.23,\$0.49 \pm \$0.13,\$0.75 \pm \$0.18,\$0.78 \pm \$0.11
Seborrheic dermatitis,\$0.16 \pm \$0.1,\$0.26 \pm \$0.15,\$0.25 \pm \$0.2,\$0.46 \pm \$0.19,\$0.81 \pm \$0.19,\$0.66 \pm \$0.12
Viral warts,\$0.03 \pm \$0.05,\$0.05 \pm \$0.08,\$0.04 \pm \$0.08,\$0.1 \pm \$0.16,\$0.9 \pm \$0.1,\$0.75 \pm \$0.09
Vitiligo,\$0.05 \pm \$0.06,\$0.09 \pm \$0.1,\$0.14 \pm \$0.16,\$0.1 \pm \$0.12,\$0.74 \pm \$0.17,\$0.83 \pm \$0.08
mean,\$0.15 \pm \$0.15,\$0.24 \pm \$0.24,\$0.29 \pm \$0.29,\$0.3 \pm \$0.3,\$0.78 \pm \$0.78,\$0.79 \pm \$0.79
